# Verify behavior cloning

Visually verify a behavior cloning based robot controller.

* Load the controller trained by the Train_BC notebook and the corresponding sp
* Choose a demonstration, to which the SP/BC pair will be applied to the images
* Visualize the path taken by the demonstration and what would be (autoregressively) recommended by the controller

In [1]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import pprint
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from bc_trainingdata import create_trainingdata_bc
from bc_factory import create_bc_model

torch.manual_seed(1)
from sensorprocessing.sp_helper import get_transform_to_sp
from sensorprocessing.sp_factory import create_sp
from demonstration.demonstration import Demonstration

from robot.al5d_position_controller import RobotPosition

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
Using device: cuda


In [ ]:
experiment = "behavior_cloning"
# run = "bc_mlp_00"
run = "bc_verify_lstm_00"

exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)

# Create the sp object described in the experiment
spexp = Config().get_experiment(exp["sp_experiment"], exp["sp_run"])
sp = create_sp(spexp, device)
transform = get_transform_to_sp(spexp)

# Load the bc model described in the experiment
bcexp = Config().get_experiment(exp["bc_experiment"], exp["bc_run"])
model, _, _ = create_bc_model(bcexp, spexp, device)
model_path = pathlib.Path(bcexp.data_dir(), bcexp["controller_file"])
model.load_state_dict(torch.load(model_path))

***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\behavior_cloning\bc_verify_lstm_00_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_verify_lstm_00 successfully loaded
Experiment:
    bc_experiment: behavior_cloning
    bc_run: bc_lstm_00
    control_size: 6
    data_dir: c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\behavior_cloning\bc_verify_lstm_00
    epochs: 10
    exp_run_sys_indep_file: C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\behavior_cloning\bc_verify_lstm_00.yaml
    experiment_name: behavior_cloning
    loss: MSELoss
    optimizer: Adam
    optimizer_lr: 0.001
    run_name: bc_verify_lstm_00
    sequence_length: 10
    sp_experiment: sensorprocessing_conv_vae
    sp_run: sp_vae_256
    subrun_name: null
    test_data:
    - - random-both-cameras
      - '2025_03_08__14_23_19'


TypeError: create_bc_model() missing 1 required positional argument: 'device'

In [ ]:
a_real_list = []
a_pred_list = []
for val in exp["verification_data"]: # for all demonstrations
    run, demo_name, camera = val
    exp_demo = Config().get_experiment("demonstration", run)
    demo = Demonstration(exp_demo, demo_name)
    for i in range(demo.metadata["maxsteps"]-1): # -1 because of lookahead
        sensor_readings, _ = demo.get_image(i, device=device, transform=transform, camera=camera)
        # FIXME: this almost surely only works for the MLP, for an LSTM 
        # I need to do the continuing stuff... generate it here. 
        # or feed it continuously
        z = sp.process(sensor_readings)        
        a_pred = model(z)
        a_real = demo.get_action(i, device=device)
        a_pred_list.append(a_pred)
        a_real_list.append(a_real)

## Plot the differences between the a_real and a_pred

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(8, 6))
titles = ["height","distance", "heading", "wrist_angle", "wrist_rotation", "gripper"]
for i in range(bcexp["control_size"]):
    ax = axs[i//3, i%3]
    ax.set_ylim(0, 1.4)
    ax.plot(t, a_real[:,i], label="ground truth")
    ax.plot(t, a_pred[:,i], label=bcexp["name"])
    if i==0:
        fig.legend(bbox_to_anchor=(1.25, 1))
    ax.set_title(titles[i])

plt.tight_layout()
graphfilename = pathlib.Path(exp.data_dir(), "bccomparison.pdf")
plt.savefig(graphfilename, bbox_inches='tight')